In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import asyncio
import itertools
from datetime import datetime
from IPython import display
import numpy as np
import pandas as pd
import pandas_ta as ta
from datetime import datetime
import vectorbt as vbt
import plotly.graph_objects as go
import mt4_hst
%matplotlib inline

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1e4
vbt.settings.portfolio["fees"] = 0.0015 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0015 # 0.25%
vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings

Portfolio Settings [Initial]


,Value
Option,
call_seq,default
init_cash,10000.0
size,100
size_type,amount
fees,0.0015
fixed_fees,0.0
slippage,0.0015
reject_prob,0.0
min_size,0.0


In [3]:
# Example Long Trends for the selected Asset
# * Uncomment others for exploration or replace them with your own TA Trend Strategy
def trends(df: pd.DataFrame):
    df['close>sma50'] = (df['close'] > df.ta.sma(50)).astype(int)
    df['ema10>ema20'] = (df.ta.ema(10) > df.ta.ema(20)).astype(int)
    donchian_20 = ta.donchian(df['high'], df['low'], lower_length=20, upper_length=20)
    donchian_50 = ta.donchian(df['high'], df['low'], lower_length=50, upper_length=50)
    df['higher_low'] = (donchian_20['DCL_20_20'] > donchian_50['DCL_50_50']).astype(int)
    return (df['close>sma50'] & df['ema10>ema20'] & df['higher_low'])
    # return ta.ma(mamode, df.close, length=fast) > ta.ma(mamode, df.close, length=slow) # SMA(fast) > SMA(slow) "Golden/Death Cross"
#     return ta.increasing(ta.ma(mamode, df.close, length=fast)) # Increasing MA(fast)
#     return ta.macd(df.close, fast, slow).iloc[:,1] > 0 # MACD Histogram is positive

In [4]:
df = mt4_hst.read_hst("stock_env/datasets/FPT1440.hst")
df = df[df['time'] >= '2012-01-01']
df.index = df['time']

In [5]:
asset_trends = trends(df)
asset_signals = df.ta.tsignals(asset_trends, asbool=True, append=True)
asset_signals.tail()

,TS_Trends,TS_Trades,TS_Entries,TS_Exits
time,,,,
2022-07-19,False,0,False,False
2022-07-20,False,0,False,False
2022-07-21,False,0,False,False
2022-07-22,False,0,False,False
2022-07-25,False,0,False,False


In [6]:
# Asset Portfolio from Trade Signals
assetpf_signals = vbt.Portfolio.from_signals(
    df.close,
    entries=asset_signals.TS_Entries,
    exits=asset_signals.TS_Exits,
)

In [7]:
# assetpf_signals.trades.plot(height=cheight, width=cwidth).show_png()

In [8]:
fig = df.close.vbt.plot(trace_kwargs=dict(name='Close'))
assetpf_signals.positions.plot(close_trace_kwargs=dict(visible=False), fig=fig)

FigureWidget({
    'data': [{'name': 'Close',
              'showlegend': True,
              'type': 'scatter…

In [9]:
values = assetpf_signals.value()
values.index = df['time']
returns = values.pct_change()

In [10]:
from stock_env.utils import create_performance
create_performance(returns)

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
values.vbt.drawdowns.plot()

In [ ]:
df.head()